In [ ]:
import numpy as np
import os
import time

from lmfit.models import GaussianModel, Model
from sklearn.linear_model import LinearRegression
from sklearn import mixture, metrics

# нужно предварительно установить geos
# pip install geos
from shapely.geometry import Polygon

from scipy.stats.distributions import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

from matplotlib import pyplot as plt
from matplotlib import cm

from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

from scipy import ndimage as ndi
from scipy.spatial import distance
from numpy import linalg

from skimage import io
from skimage.draw import ellipse
from skimage.color import rgb2gray
from skimage import filters
from skimage.morphology import disk

from PIL import Image, ImageDraw, ImageFilter, ImageOps
import copy
import cv2

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import distance_transform_edt as edt
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon

from radio_beam.commonbeam import getMinVolEllipse

from utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology


## Загружаем изображения

In [ ]:
def img_load_preprocess(path, img_names, h=150, k=1):
    imgs = []
    for i, class_img_names in enumerate(img_names):
        imgs.append([])
        for name in class_img_names:
            img = io.imread(path + '/' + name)
            img = grainPreprocess.image_preprocess(img, h, k)
            imgs[i].append(img)
    return imgs

In [ ]:
path = 'images'
grain_names = np.array(
    [['Ultra_Co8.jpg'], ['Ultra_Co11.jpg'], ['Ultra_Co6_2.jpg'], ['Ultra_Co15.jpg'], ['Ultra_Co25.jpg']])
grain_images = img_load_preprocess(path, grain_names)

## Распределение углов

In [ ]:
def angles_legend(name, itype, mus, sigmas, amps, norm, ):
    #
    # создание легенды распределения углов
    #

    mu1 = round(mus[0], 2)
    sigma1 = round(sigmas[0], 2)
    amp1 = round(amps[0], 2)

    mu2 = round(mus[1], 2)
    sigma2 = round(sigmas[1], 2)
    amp2 = round(amps[1], 2)

    val = round(norm, 4)

    border = '--------------\n'
    total_number = '\n количество углов ' + str(val)
    text_angle = '\n шаг угла ' + str(step) + ' градусов'

    moda1 = '\n mu1 = ' + str(mu1) + ' sigma1 = ' + str(sigma1) + ' amp1 = ' + str(amp1)
    moda2 = '\n mu2 = ' + str(mu2) + ' sigma2 = ' + str(sigma2) + ' amp2 = ' + str(amp2)

    legend = border + name + ' ' + itype + total_number + text_angle + moda1 + moda2

    return legend


### Посчитаем углы для всех образцов и аппроксимируем при помощи бимодального гаусса

In [ ]:
def angles_approx_save(folder, images, names, types, step, save=False):
    #
    # вывод распределения углов для всех фотографий одного образца
    #

    texts = []
    xy_scatter = []
    xy_gauss = []

    if not os.path.exists(folder):
        os.mkdir(folder)

    for i, images_list in enumerate(images):
        all_original_angles = []

        for j, image in enumerate(images_list):
            original_angles = grainMark.get_angles(image)

            for angle in original_angles:
                all_original_angles.append(angle)

        angles, angles_set, dens_curve = grainStats.stats_preprocess(all_original_angles, step)

        x = angles_set.astype(np.float64)
        y = dens_curve

        norm = np.sum(y)
        y = y / norm

        (x_gauss, y_gauss), mus, sigmas, amps = grainApprox.bimodal_gauss_approx(x, y)

        text = angles_legend(names[i], types[i], mus, sigmas, amps, norm)

        xy_gauss.append((x_gauss, y_gauss))
        xy_scatter.append((x, y))

        texts.append(text)

    if save:
        np.save(f'{folder}/xy_scatter_step_{step}.npy', np.array(xy_scatter, dtype=object))
        np.save(f'{folder}/xy_gauss_step_{step}.npy', np.array(xy_gauss))
        np.save(f'{folder}/texts_step_{step}.npy', np.array(texts))


In [ ]:
all_images = np.load('saved np/all_images.npy')


### Считаем все изображения, предобработаем их и сохраним

In [ ]:
images_dir = 'data'

folders_names = os.listdir(images_dir)
images_paths_raw = [os.listdir(images_dir + '/' + folder) for folder in folders_names]

images_paths = []
for i, folder in enumerate(images_paths_raw):
    images_paths.append([])
    for image_path in folder:
        images_paths[i].append(images_dir + '/' + folders_names[i] + '/' + image_path)

all_images = []
for images_folder in images_paths:
    images = [grainPreprocess.image_preprocess(io.imread(name)) for i, name in enumerate(images_folder) if i <= 100]
    all_images.append(images)

In [ ]:
% matplotlib widget
all_images = np.load('saved np/all_images.npy')
folder = 'saved np'
grain_names = np.array(
    [['Ultra_Co8.jpg'], ['Ultra_Co11.jpg'], ['Ultra_Co6_2.jpg'], ['Ultra_Co15.jpg'], ['Ultra_Co25.jpg']])
types = ['средние зерна', 'мелкие зерна', 'мелкие зерна', 'крупные зерна', 'средне-мелкие зерна']

for step in range(1, 6):
    # генерация распределения, аппроксимирование распределения и сохранение данных
    angles_approx_save(folder, all_images, grain_names[:, 0], types, step)

# np.save('all_images.npy', all_images)

In [ ]:
def angles_approx(folder, step, width, height, font_size=20):
    #
    # вывод распределения углов для всех фотографий одного образца
    #

    xy_scatter = np.load(f'{folder}/xy_scatter_step_{step}.npy', allow_pickle=True)
    xy_gauss = np.load(f'{folder}/xy_gauss_step_{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/texts_step_{step}.npy', allow_pickle=True)

    plt.rcParams['font.size'] = '15'
    gs = gridspec.GridSpec(1, 1)
    plt.figure(figsize=(width, height))

    ax = plt.subplot(gs[0])
    # маркеры для одновременного вывода скаттера для разных классов
    # количество варкеров=количество классов-1
    markers = ['v', 's', 'D', 'd', 'p', '*']
    for i in range(len(names)):
        marker = markers[i]
        ax.plot(xy_gauss[i][0], xy_gauss[i][1])
        ax.scatter(xy_scatter[i][0], xy_scatter[i][1], marker=marker)

    ax.legend(texts, fontsize=14, loc=1)
    plt.ylabel('p(x)', fontsize=font_size)
    plt.xlabel('угол связующего вещества, градусы', fontsize=font_size)

    #  plt.savefig('распределение углов')
    plt.show()

In [ ]:
folder = 'saved np'
step = 5
width = 20
height = 15

# генерация распределения, аппроксимирование распределения и сохранение данных
angles_approx(folder, step, width, height)

In [ ]:
def angles_approx_tail(start, stop, M, font_size=25):
    #
    # линейная аппроксимация конца
    #
    xy_scatter_o = np.load(f'{folder}/xy_scatter_step_{step}.npy', allow_pickle=True)
    xy_gauss = np.load(f'{folder}/xy_gauss_step_{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/texts_step_{step}.npy', allow_pickle=True)

    xy_scatter = []

    for j, sc in enumerate(xy_scatter_o):
        x = sc[0]
        y = sc[1]
        xy_scatter.append([[], []])
        for i, xi in enumerate(x):
            if xi <= stop and xi >= start:
                xy_scatter[j][0].append(x[i])
                xy_scatter[j][1].append(y[i])

    markers = ['v', 's', 'D', 'd', 'p', '*']
    k = 2
    for i in range(len(xy_scatter_o)):
        plt.figure(figsize=(M, M))
        marker = markers[i]
        plt.plot(xy_gauss[i, start:stop])
        #  print(xy_scatter[i])

        x = np.array(xy_scatter[i][0][:-1])
        y = np.array(xy_scatter[i][1][:-1])

        #     x=abs(x)

        x = x[1:].reshape(-1, 1)
        y = y[1:].reshape(-1, 1)

        #    x=np.log(x)
        #    y=np.log(y)

        plt.scatter(x, y, marker=marker)

        (x_pred, y_pred), k, b, angle, score = grainApprox.lin_regr_approx(x, y)

        #   start1=np.log(360-start)
        #  stop1=np.log(360-stop)
        stop1 = stop
        start1 = start

        diff = stop1 - start1

        #   x_positions=np.linspace(start1,stop1,8)
        #   x_labels=np.linspace(diff,0,8)
        #   x_labels=np.round(x_labels,1).astype('str')
        #   x_labels=list(x_labels)

        border = '--------------'

        lin_k = '\n k наклона ' + str(round((k), 3)) + ' сдвиг b ' + str(round(b, 3))
        lin_k_angle = '\n угол наклона $' + str(round(angle, 3)) + '^{\circ}$'
        acc = '\n точность ' + str(round(score, 2))
        legend = lin_k + lin_k_angle + acc
        plt.plot(x_pred, y_pred)

        plt.legend([legend, texts[i]], fontsize=14, loc=1)
        plt.ylabel('ln(p(x))', fontsize=font_size)
        plt.xlabel('угол св-ва ln($360-\phi$ ), градусы', fontsize=font_size)
        #    plt.xticks(x_positions,x_labels)

        # plt.ylim([0,max(np.log(xy_scatter[i][1][:-1]))])
        #   plt.savefig('хвост распределения ln-ln  '+names[i])

        #  print(texts)

        #  plt.savefig('распределение углов')
        plt.show()

In [ ]:
M = 15
N = 35
step = 1

start = 340
stop = 360

angles_approx_tail(start, stop, M)

In [ ]:
def angles_approx_modes(folder,step,start1, stop1, start2, stop2, width, height, font_size=25):
    #
    # аппроксимация каждой моды
    #
    xy_scatter_o = np.load(f'{folder}/xy_scatter_step_{step}.npy', allow_pickle=True)
    xy_gauss_o = np.load(f'{folder}/xy_gauss_step_{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/texts_step_{step}.npy', allow_pickle=True)

    xy_scatter1 = []
    xy_scatter2 = []

    for j, sc in enumerate(xy_scatter_o):
        x = sc[0]
        y = sc[1]
        xy_scatter1.append([[], []])
        xy_scatter2.append([[], []])
        for i, xi in enumerate(x):
            if xi <= stop1 and xi >= start1:
                xy_scatter1[j][0].append(x[i])
                xy_scatter1[j][1].append(y[i])

            if xi <= stop2 and xi >= start2:
                xy_scatter2[j][0].append(x[i])
                xy_scatter2[j][1].append(y[i])

    plt.rcParams['font.size'] = '25'
    markers = ['v', 's', 'D', 'd', 'p', '*']
    k = 2
    for i in range(len(xy_scatter_o)):
        plt.figure(figsize=(width, height))
        marker = markers[i]

        #    plt.scatter(xy_scatter_o[i][0],xy_scatter_o[i][1])

        x1 = np.array(xy_scatter1[i][0])
        y1 = xy_scatter1[i][1]

        x2 = np.array(xy_scatter2[i][0])
        y2 = xy_scatter2[i][1]

        mu1, sigma1, amp1 = grainApprox.gaussian_fit(x1, y1, mu=120)
        mu2, sigma2, amp2 = grainApprox.gaussian_fit(x2, y2, mu=225)

        x_gauss1 = np.arange(mu1 - 3 * sigma1, mu1 + 3 * sigma1)
        x_gauss2 = np.arange(mu2 - 3 * sigma2, mu2 + 3 * sigma2)

        y_gauss1 = grainStats.gaussian(x_gauss1, mu1, sigma1, amp1)
        y_gauss2 = grainStats.gaussian(x_gauss2, mu2, sigma2, amp2)

        #    plt.plot(x_gauss,y_gauss)
        plt.scatter(x1, y1, marker=marker)
        plt.plot(x_gauss1, y_gauss1)

        plt.scatter(x2, y2, marker=markers[i + 1])
        plt.plot(x_gauss2, y_gauss2)

        #  print('mu1=',mu1,' sigma1=',sigma1,'amp1=',amp1)
        #  print('mu2=',mu2,' sigma2=',sigma2,'amp2=',amp2)
        plt.legend(['moda1', 'moda2', 'approx1', texts[i]], fontsize=14, loc=1)
        plt.ylabel('p(x)', fontsize=font_size)
        plt.xlabel('угол связующего вещества, градусы', fontsize=font_size)
        # plt.ylim([0,max(np.log(xy_scatter[i][1][:-1]))])
    # plt.savefig('крупные моды отдельная аппрокс  ' + names[i])

    #  print(texts)

    #  plt.savefig('распределение углов')
    plt.show()

In [ ]:
folder='saved np'
width = 12
height = 15

step = 1

start = 340
stop = 360

start1 = 50
stop1 = 180

start2 = 180
stop2 = 300

angles_approx_modes(folder,step, start1, stop1, start2, stop2, width, height)

## Распределение диаметров

https://radio-beam.readthedocs.io/en/latest/api/radio_beam.commonbeam.getMinVolEllipse.html#radio_beam.commonbeam.getMinVolEllipse

In [ ]:
def beams_legend(name, itype, norm, k, angle, b, score, dist_step, dist_mean):
    #
    # создание легенды для распределения длин полуосей
    #
    border = '--------------'
    tp = '\n ' + name + ' тип ' + itype
    num = '\n регионы Co ' + str(norm) + ' шт'
    lin_k = '\n k наклона ' + str(round((k), 3)) + ' сдвиг b ' + str(round(b, 3))
    lin_k_angle = '\n угол наклона $' + str(round(angle, 3)) + '^{\circ}$'
    acc = '\n точность ' + str(round(score, 2))
    text_step = '\n шаг длины ' + str(dist_step) + '$ мкм$'
    mean_text = '\n средняя длина ' + str(round(dist_mean, 2))
    legend = border + tp + lin_k + lin_k_angle + acc + num + text_step + mean_text

    return legend

In [ ]:
def diametr_approx(images, names, types, step, pixel, N, M):
    #
    # вывод распределения длин а- и б- полуосей для разных образцов
    #
    texts1 = []
    texts2 = []

    xy_scatter1 = []
    xy_scatter2 = []

    xy_pred1 = []
    xy_pred2 = []

    for j, image in enumerate(images):
        a_beams, b_beams, angles, cetroids = grainMark.get_mvee_params(image, 0.2)

        distances1, dist1_set, dens1_curve = grainStats.stats_preprocess(a_beams, step)
        distances2, dist2_set, dens2_curve = grainStats.stats_preprocess(b_beams, step)
        angles, angles_set, angles_dens_curve = grainStats.stats_preprocess(np.rad2deg(angles).astype('int32'), step=5)

        norm1 = round(np.sum(dens1_curve), 6)
        norm2 = round(np.sum(dens2_curve), 6)

        x1 = np.array([dist1_set]).reshape(-1, 1) * pixel
        y1 = np.log([dens1_curve / norm1]).reshape(-1, 1)

        x2 = np.array([dist2_set]).reshape(-1, 1) * pixel
        y2 = np.log([dens2_curve / norm2]).reshape(-1, 1)

        x1 = x1[2:-3]
        y1 = y1[2:-3]

        x2 = x2[2:-3]
        y2 = y2[2:-3]

        (x_pred1, y_pred1), k1, b1, angle1, score1 = lin_regr_approx(x1, y1)
        (x_pred2, y_pred2), k2, b2, angle2, score2 = lin_regr_approx(x2, y2)

        dist_step = pixel * step

        legend1 = beams_legend(names[j], types[j], norm1, k1, angle1, b1, score1, dist_step, distances1.mean() * pixel)
        legend2 = beams_legend(names[j], types[j], norm2, k2, angle2, b2, score2, dist_step, distances2.mean() * pixel)

        texts1.append(legend1)
        texts2.append(legend2)

        xy_scatter1.append((x1, y1))
        xy_scatter2.append((x2, y2))

        xy_pred1.append((x_pred1, y_pred1))
        xy_pred2.append((x_pred2, y_pred2))

    plt.rcParams['font.size'] = '15'
    gs = gridspec.GridSpec(2, 1)
    # для статьи 12,18
    plt.figure(figsize=(N, M))

    ax1 = plt.subplot(gs[0])
    ax2 = plt.subplot(gs[1])

    ax1.set_title('распределение a-полуосей связующего вещества', fontsize=15)
    ax2.set_title('распределение b-полуосей связующего вещества', fontsize=15)

    ax1.set_ylabel('ln(p(x))', fontsize=15)
    ax1.set_xlabel('длина полуоси, мкм', fontsize=15)

    ax2.set_ylabel('ln(p(x))', fontsize=15)
    ax2.set_xlabel('длина полуоси, мкм', fontsize=15)

    #  ax3.grid()
    markers = ['v', 's', 'D', 'd', 'p', '*']
    for i in range(len(images)):
        marker = markers[i]
        ax1.plot(xy_pred1[i][0], xy_pred1[i][1])
        ax1.scatter(xy_scatter1[i][0], xy_scatter1[i][1], marker=marker)

        ax2.plot(xy_pred2[i][0], xy_pred2[i][1])
        ax2.scatter(xy_scatter2[i][0], xy_scatter2[i][1], marker=marker)

    #  plt.savefig('распределение полуосей')
    #  print(texts1)
    #  print(texts2)

    ax1.legend(texts1, fontsize=14, loc=1)
    ax2.legend(texts1, fontsize=14, loc=1)

    plt.show()

In [ ]:
N = 20
M = 20
# для статье N=12,M=18
pixel = 49.7 / 1000
step = 5

images = grain_images[:4]
names = grain_names[:4]
types = ['мелкие зерна', ' средние зерна', ' крупные зерна', 'some grain', 'some grain']

diametr_approx(images, names, types, step, pixel, N, M)

### Диаметры для всех образцов

In [ ]:
def diametr_approx_all(images, names, types, step, pixel, N, M):
    #
    # вывод распределения длин а- и б- полуосей для разных образцов
    #
    texts1 = []
    texts2 = []

    xy_scatter1 = []
    xy_scatter2 = []

    xy_pred1 = []
    xy_pred2 = []

    for i, images_list in enumerate(images):
        all_a_beams = []
        all_b_beams = []

        for j, image in enumerate(images_list):
            try:
                a_beams, b_beams, angles, cetroids = grainMark.get_mvee_params(image, 0.2)
                for k in range(len(a_beams)):
                    all_a_beams.append(a_beams[k])
                    all_b_beams.append(b_beams[k])
            except Exception:
                pass
                #print('error i=',i,' j=',j)

        distances1, dist1_set, dens1_curve = grainStats.stats_preprocess(all_a_beams, step)
        distances2, dist2_set, dens2_curve = grainStats.stats_preprocess(all_b_beams, step)

        angles, angles_set, angles_dens_curve = grainStats.stats_preprocess(np.rad2deg(angles).astype('int32'), step=5)

        norm1 = round(np.sum(dens1_curve), 6)
        norm2 = round(np.sum(dens2_curve), 6)

        x1 = np.array([dist1_set]).reshape(-1, 1) * pixel
        y1 = np.log([dens1_curve / norm1]).reshape(-1, 1)

        x2 = np.array([dist2_set]).reshape(-1, 1) * pixel
        y2 = np.log([dens2_curve / norm2]).reshape(-1, 1)

        x1 = x1[2:-3]
        y1 = y1[2:-3]

        x2 = x2[2:-3]
        y2 = y2[2:-3]

        (x_pred1, y_pred1), k1, b1, angle1, score1 = lin_regr_approx(x1, y1)
        (x_pred2, y_pred2), k2, b2, angle2, score2 = lin_regr_approx(x2, y2)

        dist_step = pixel * step

        legend1 = beams_legend(names[i], types[i], norm1, k1, angle1, b1, score1, dist_step, distances1.mean() * pixel)
        legend2 = beams_legend(names[i], types[i], norm2, k2, angle2, b2, score2, dist_step, distances2.mean() * pixel)

        texts1.append(legend1)
        texts2.append(legend2)

        xy_scatter1.append((x1, y1))
        xy_scatter2.append((x2, y2))

        xy_pred1.append((x_pred1, y_pred1))
        xy_pred2.append((x_pred2, y_pred2))

    plt.rcParams['font.size'] = '20'
    gs = gridspec.GridSpec(2, 1)
    # для статьи 12,18
    plt.figure(figsize=(N, M))

    ax1 = plt.subplot(gs[0])
    ax2 = plt.subplot(gs[1])

    ax1.set_title('распределение a-полуосей связующего вещества', fontsize=20)
    ax2.set_title('распределение b-полуосей связующего вещества', fontsize=20)

    ax1.set_ylabel('ln(p(x))', fontsize=15)
    ax1.set_xlabel('длина полуоси, мкм', fontsize=15)

    ax2.set_ylabel('ln(p(x))', fontsize=15)
    ax2.set_xlabel('длина полуоси, мкм', fontsize=15)

    #  ax3.grid()
    markers = ['v', 's', 'D', 'd', 'p', '*']
    for i in range(len(images)):
        marker = markers[i]
        ax1.plot(xy_pred1[i][0], xy_pred1[i][1])
        ax1.scatter(xy_scatter1[i][0], xy_scatter1[i][1], marker=marker)

        ax2.plot(xy_pred2[i][0], xy_pred2[i][1])
        ax2.scatter(xy_scatter2[i][0], xy_scatter2[i][1], marker=marker)

    #  plt.savefig('распределение полуосей')
    #  print(texts1)
    #  print(texts2)

    ax1.legend(texts1, fontsize=14, loc=1)
    ax2.legend(texts2, fontsize=14, loc=1)

    plt.show()

In [ ]:
folders_names

In [ ]:
N = 30
M = 30
# для статье N=12,M=18
pixel = 49.7 / 1000
step = 5

images = all_images
names = folders_names
types = ['средние зерна', 'мелкие зерна', 'мелкие зерна', 'крупные зерна', 'средне-мелкие зерна']

diametr_approx_all(images, names, types, step, pixel, N, M)

In [ ]:
#grainMark.get_mvee_params(images[0][15],0.2)

In [ ]:
def draw_edges(image):
    #
    # рисует на изображении линии по точкам контура cnts
    # линии в стиле x^1->x^2,x^2->x^3 и тд
    #
    new_image = copy.copy(image)
    im = Image.fromarray(np.uint8(cm.gist_earth(new_image) * 255))
    draw = ImageDraw.Draw(im)
    width = 50

    cnts = grainMark.get_contours(image)

    for i, cnt in enumerate(cnts):
        if len(cnt) > 2:

            for i, point in enumerate(cnt):
                if i < len(cnt) - 1:
                    point1 = cnt[i - 1]
                    point2 = cnt[i]
                    point3 = cnt[i + 1]
                    x1, y1 = point1[1], point1[0]
                    x2, y2 = point2[1], point2[0]
                    x3, y3 = point3[1], point3[0]

                    v1 = np.array((x1 - x2, y1 - y2)).reshape(1, 2)
                    v2 = np.array((x3 - x2, y3 - y2)).reshape(1, 2)

                    dot = np.dot(v1[0], v2[0])
                    dist1 = np.linalg.norm(v1[0])
                    dist2 = np.linalg.norm(v2[0])
                    cos = dot / (dist1 * dist2)

                    v = np.concatenate([v1, v2])
                    det = np.linalg.det(v)

                    angle = 0
                    if abs(cos) < 1:
                        ang = int(np.arccos(cos) * 180 / np.pi)
                        if det < 0:
                            angle = ang
                        else:
                            angle = 360 - ang

                    p2 = point

                    x2, y2 = p2[1], p2[0]

                    color = (150, 150, 150)

                    line_color = (0, 255, 127)
                    if angle <= 25:
                        color = (0, 0, 255)
                        r = 4
                        draw.ellipse((y2 - r, x2 - r, y2 + r, x2 + r), fill=color, width=5)
                        draw.line((point1[0], point1[1], point2[0], point2[1]), fill=line_color, width=4)
                        draw.line((point3[0], point3[1], point2[0], point2[1]), fill=line_color, width=4)
                        img = np.array(im.convert('RGB'))

                        cropped = np.zeros((width + 1, width + 1, 3), dtype='uint8')

                        #         print('img shape=',img.shape)
                        for i, xi in enumerate(range(x2 - width - 1, x2 + width - 2)):
                            for j, yi in enumerate(range(y2 - width - 1, y2 + width - 2)):
                                if xi < img.shape[0] and yi < img.shape[1] and i < width and j < width:
                                    #            print('xi=',xi,' yi=',yi)
                                    #             print('val=',img[xi,yi],'\n------------')

                                    cropped[i, j] = img[xi, yi]
                        #      plt.figure(figsize=(15,15))
                        #   plt.imshow(cropped)
                        #    plt.imshow(im)
                        #       plt.show()
                        #    res = cv2.resize(cropped, dsize=(500, 500), interpolation=cv2.INTER_CUBIC)
                        io.imsave('small angles/' + str(time.time()) + '.png', cropped)


                    elif angle >= 335:
                        color = (255, 0, 0)
                        r = 4
                        draw.ellipse((y2 - r, x2 - r, y2 + r, x2 + r), fill=color, width=5)
                        draw.line((point1[0], point1[1], point2[0], point2[1]), fill=line_color, width=4)
                        draw.line((point3[0], point3[1], point2[0], point2[1]), fill=line_color, width=4)

                        img = np.array(im.convert('RGB'))

                        cropped = np.zeros((width + 1, width + 1, 3), dtype='uint8')

                        #         print('img shape=',img.shape)
                        for i, xi in enumerate(range(x2 - width - 1, x2 + width - 2)):
                            for j, yi in enumerate(range(y2 - width - 1, y2 + width - 2)):
                                if xi < img.shape[0] and yi < img.shape[1] and i < width and j < width:
                                    #            print('xi=',xi,' yi=',yi)
                                    #             print('val=',img[xi,yi],'\n------------')

                                    cropped[i, j] = img[xi, yi]
                        #  plt.figure(figsize=(15,15))
                        #   plt.imshow(cropped)
                        #    plt.imshow(im)
                        #  plt.show()
                        io.imsave('small angles/' + str(time.time()) + '.png', cropped)

        else:
            continue

    img = np.array(im)

    return img

In [ ]:
marked_images = [draw_edges(image) for image in grain_images]


In [ ]:
for i, img in enumerate(marked_images):
    io.imsave(f'low angles {i}.png', img)

In [ ]:
grainShow.img_show(marked_images[1])